In [1]:
# from IPython.display import Audio

import librosa
from glob import glob
import librosa.display
import numpy as np
import librosa

import matplotlib.pyplot as plt
import os
import shutil

In [38]:
#https://stackoverflow.com/questions/2472552/python-way-to-clone-a-git-repository/34424094
#https://stackoverflow.com/questions/15315573/how-can-i-call-git-pull-from-within-python
#https://stackoverflow.com/questions/2472552/python-way-to-clone-a-git-repository
#https://github.com/jmoiron/python-github
def download_data(link):
  folder_name = link.split('/')[4].split('.git')[0]
  if os.path.exists(folder_name):
    print('remove ', folder_name)
    shutil.rmtree(folder_name)
    os.system(f'git clone {link}')
  else:
    print('download')
    os.system(f'git clone {link}')

def get_folder_name(link):
  return link.split('/')[4].split('.git')[0]

In [39]:
link = 'https://github.com/ming0520/fyp2-data-20201126-fortrain.git'
download_data(link)

download


In [40]:
link = 'https://github.com/ming0520/fyp2-data-20201126-fortrain.git'
folder_name = get_folder_name(link)
print(folder_name)

fyp2-data-20201126-fortrain


In [41]:
#Data Augmentation
# https://medium.com/@keur.plkar/audio-data-augmentation-in-python-a91600613e47
# https://gist.github.com/keyurparalkar/5a49f696ed36ddce6526ab50e29e04ce
# https://pypi.org/project/audiomentations/
# https://www.kaggle.com/CVxTz/audio-data-augmentation
# https://www.kaggle.com/haqishen/augmentation-methods-for-audio
# https://www.tensorflow.org/io/tutorials/audio

In [42]:
DATA_PATH = folder_name
NORMAL_AUDIOS = glob(f'./{DATA_PATH}/success/*.wav')
STUTTERED_AUDIOS = glob(f'./{DATA_PATH}/stuttered/*.wav')
failed_list = []
# print(NORMAL_AUDIOS)
# print(STUTTERED_AUDIOS)

In [43]:
print(len(NORMAL_AUDIOS))
print(len(STUTTERED_AUDIOS))
print(STUTTERED_AUDIOS[2141].split('/')[1].split('.wav')[0])
print(NORMAL_AUDIOS[100].split('/')[1].split('.wav')[0])

3492
3430
fyp2-data-20201126-fortrain
fyp2-data-20201126-fortrain


In [12]:
class FeatureExtraction:
    def __init__(self, n_mels=128):
        self.n_mels = n_mels
        self.y = None
        self.sr = 11050
        self.S = None
        self.log_S = None
        self.mfcc = None
        self.delta_mfcc = None
        self.delta2_mfcc = None
        self.M = None
        self.rmse = None
        self.foldername = None
        self.filename=None
    
    def loadFile(self, foldernname):
        self.foldernname=foldernname
        self.y, self.sr = librosa.load(foldernname)
#         logger.debug('File loaded: %s', foldernname)
    
    def load_y_sr(self, y, sr):
        self.y = y
        self.sr = sr
    
    def melspectrogram(self):
        self.S = librosa.feature.melspectrogram(self.y, sr=self.sr, n_mels=self.n_mels)
        self.log_S = librosa.amplitude_to_db(self.S)
    
    def plotmelspectrogram(self, save=True):
        fig = plt.figure(figsize=(12, 4))
        librosa.display.specshow(self.log_S, sr=self.sr, x_axis='time', y_axis='mel')
        plt.title(f'mel Power Spectrogram ({self.filename})')
        plt.colorbar(format='%+02.0f dB')
        plt.tight_layout()
        if not os.path.exists('mel'):
          os.mkdir('mel')
        if save:
          fig.savefig(f'./mel/{self.filename}-mel.png', dpi=fig.dpi)
          print(f'Saved to ./mel/{self.filename}-mel.png')
          plt.close('all')

    def extractmfcc(self, n_mfcc=13):
        self.mfcc = librosa.feature.mfcc(S=self.log_S, n_mfcc=n_mfcc)
        self.delta_mfcc = librosa.feature.delta(self.mfcc,mode='nearest')
        self.delta2_mfcc = librosa.feature.delta(self.mfcc, order=2,mode='nearest')
        self.M = np.vstack([self.mfcc, self.delta_mfcc, self.delta2_mfcc])
    
    def plotmfcc(self,save=False):
        fig = plt.figure(figsize=(12, 6))
        plt.subplot(3, 1, 1)
        librosa.display.specshow(self.mfcc)
        plt.title(f'mel Power Spectrogram ({self.filename})')
        plt.ylabel('MFCC')
        plt.colorbar()
        
        plt.subplot(3, 1, 2)
        librosa.display.specshow(self.delta_mfcc)
        plt.title(f'mel Power Spectrogram ({self.filename})')
        plt.ylabel('MFCC-$\Delta$')
        plt.colorbar()
        
        plt.subplot(3, 1, 3)
        librosa.display.specshow(self.delta2_mfcc, sr=self.sr, x_axis='time')
        plt.title(f'mel Power Spectrogram ({self.filename})')
        plt.ylabel('MFCC-$\Delta^2$')
        plt.colorbar()
        
        plt.tight_layout()
        if not os.path.exists('mfcc'):
          os.mkdir('mfcc')
        if save:
          fig.savefig(f'./mfcc/{self.filename}-mfcc.png', dpi=fig.dpi)
          print(f'Saved to ./mfcc/{self.filename}-mfcc.png')
          plt.close('all')

    def extractrmse(self):
        self.rmse = librosa.feature.rms(y=self.y)

In [13]:
folder_name = '20201201/20201201-all'

In [14]:
print(os.popen(f'ls {folder_name}').read())

stuttered
success



In [9]:
for i in range(10):
  feature = FeatureExtraction()
  feature.loadFile(NORMAL_AUDIOS[i])
  feature.melspectrogram()
  feature.extractmfcc()
  feature.extractrmse()
  print("mfcc Shape: ",feature.mfcc.shape)

NameError: name 'NORMAL_AUDIOS' is not defined

In [47]:
print("mfcc Type: ",type(feature.mfcc))
print("mfcc Shape: ",feature.mfcc.shape)
print("delta_mfcc Type: ",type(feature.delta_mfcc))
print("delta_mfcc Shape: ",feature.delta_mfcc.shape)
print("delta2_mfcc Type: ",type(feature.delta2_mfcc))
print("delta2_mfcc Shape: ",feature.delta2_mfcc.shape)
print("length mfcc :", len(feature.mfcc)) 
print("length delta_mfcc :", len(feature.delta_mfcc)) 
print("length delta2_mfcc :", len(feature.delta2_mfcc)) 

mfcc Type:  <class 'numpy.ndarray'>
mfcc Shape:  (13, 19)
delta_mfcc Type:  <class 'numpy.ndarray'>
delta_mfcc Shape:  (13, 19)
delta2_mfcc Type:  <class 'numpy.ndarray'>
delta2_mfcc Shape:  (13, 19)
length mfcc : 13
length delta_mfcc : 13
length delta2_mfcc : 13


In [48]:
for fea in feature.mfcc:
  print(fea)

[-637.66943282 -630.20868172 -640.44335342 -644.1877008  -647.16255658
 -639.46350713 -608.78443161 -612.74994604 -615.48376443 -615.04251755
 -613.46534004 -620.52567875 -623.21736037 -626.238512   -640.50267237
 -660.53504156 -669.11258542 -678.89964529 -685.97663936]
[ 88.58059851  98.07392275  84.72124267  79.00970677  74.60212156
  81.14329346 105.64582818 106.3159523   96.35278586  99.50476701
  98.15260525  95.38899635  97.22047571  98.79454445  82.55541191
  58.47047722  47.2564368   33.77049823  23.87363951]
[78.45725831 85.22522289 74.10861885 68.26314633 63.50854839 68.63656552
 80.115157   68.06443109 53.03588077 55.58610416 51.31529118 53.68729274
 66.26670191 76.53935844 68.65784327 54.01697466 45.06632711 32.55913694
 22.99000972]
[64.68627542 68.01019343 59.05523313 54.58806718 49.84374278 52.91523475
 56.29791527 37.31729371 26.89091055 23.08431386 19.72318655 21.95555227
 36.97358555 53.78192922 53.36187333 48.12653509 41.73131465 30.61692092
 21.57201661]
[48.2178402

In [13]:
import json
import os
import math
import librosa

class Dataset:
  def __init__(self,dataset_path,output_path):
    self.DATASET_PATH = dataset_path
    self.OUTPUT_PATH = output_path
    self.X = np.empty(shape=(0,80))
    self.Y = np.empty(shape=(0,2))
    self.DATASET = None
    self.PATH_ARRAY = []
    # self.TRACK_DURATION
    # self.SAMPLES_PER_TRACK
    print("Object created!")
    self.process_dataset()
    self.__write_to_file()

  def process_dataset(self):
    processed_counter = 0
    failed_file = []
    unexpected_label = []
    for i , (dirpath, dirnames, filenames) in enumerate(os.walk(self.DATASET_PATH)):
      if dirpath is not self.DATASET_PATH:
        label = dirpath.split("/")[-1]
        # print(label)
        print("Processing:", label)
        for file in filenames:
          #load audio
          file_path = os.path.join(dirpath,file)
          # print(file_path)

          #exctract mfcc
          try:
            features = FeatureExtraction()
            features.loadFile(file_path)
            features.melspectrogram()
            features.extractmfcc()
            features.extractrmse()
          except ValueError:
            failed_file.apppend(file_path)
          
          feature_vector = []

          for feature in features.mfcc:
            feature_vector.append(np.mean(feature))
            feature_vector.append(np.var(feature))
          
          for feature in features.delta_mfcc:
            feature_vector.append(np.mean(feature))
            feature_vector.append(np.var(feature))

          for feature in features.delta2_mfcc:
            feature_vector.append(np.mean(feature))
            feature_vector.append(np.var(feature))

          feature_vector.append(np.mean(features.rmse))
          feature_vector.append(np.var(features.rmse))

          self.X = np.vstack((self.X,[feature_vector]))
          if label == 'success':
            self.Y = np.vstack((self.Y,[0,1]))
            processed_counter += 1
          elif label == 'stuttered':
            self.Y = np.vstack((self.Y,[1,0]))
            processed_counter += 1
          else:
            unexpected_label.append(file_path)
          print("Done ", processed_counter, file_path)

    for fail in unexpected_label:
      print("unexpected_label ", file_path, " !")
    
    for fail in failed_file:
      print("fail ", file_path, " !")

    print("finished all!")    
    self.DATASET = np.hstack((X,Y))      
    
  def __write_to_file(self):
    if os.path.exists(self.OUTPUT_PATH):
      os.remove(self.OUTPUT_PATH)

    np.savetxt(self.OUTPUT_PATH, self.DATASET)

In [ ]:
if os.path.exists('fyp2-data-20201126-fortrain/.git'):
  os.system('rm -rf fyp2-data-20201126-fortrain/.git')

In [ ]:
ds = Dataset('train2_augmented_normalize_11050hz','train.gz')

All under is the prototype for dataset class

In [60]:
# data = {
#     "mapping": [],
#     "labels": [],
#     "data":[]
# }

# dataset_path = 'fyp2-data-20201126-fortrain'
# output_file = 'dataset.gz'

# X = np.empty(shape=(0,80))
# Y = np.empty(shape=(0,2))

# failed_file = []
# unexpected_label = []


In [52]:
# #exctract mfcc
# try:
#   features = FeatureExtraction()
#   features.loadFile(file_path)
#   features.melspectrogram()
#   features.extractmfcc()
#   features.extractrmse()
# except ValueError:
#   failed_file.apppend(file_path)

# feature_vector = []

# for feature in features.mfcc:
#   feature_vector.append(np.mean(feature))
#   feature_vector.append(np.var(feature))

# for feature in features.delta_mfcc:
#   feature_vector.append(np.mean(feature))
#   feature_vector.append(np.var(feature))

# for feature in features.delta2_mfcc:
#   feature_vector.append(np.mean(feature))
#   feature_vector.append(np.var(feature))

# feature_vector.append(np.mean(features.rmse))
# feature_vector.append(np.var(features.rmse))

# print(len(feature_vector))
# X = np.vstack((X,[feature_vector]))
# if label == 'success':
#   Y = np.vstack((Y,[0,1]))
#   processed_counter += 1
# elif label == 'stuttered':
#   Y = np.vstack((Y,[1,0]))
#   processed_counter += 1
# else:
#   unexpected_label.append(file_path)
# print("Done ", file_path)


In [62]:
# processed_counter = 0

# for i , (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
#   if dirpath is not dataset_path:
#     label = dirpath.split("/")[-1]
#     data['mapping'] = label
#     # print(label)
#     print("Processing:", label)
#     for file in filenames:
#       if processed_counter == 10:
#         break
#       #load audio
#       file_path = os.path.join(dirpath,file)
#       # print(file_path)

#       #exctract mfcc
#       try:
#         features = FeatureExtraction()
#         features.loadFile(file_path)
#         features.melspectrogram()
#         features.extractmfcc()
#         features.extractrmse()
#       except ValueError:
#         failed_file.apppend(file_path)
      
#       feature_vector = []

#       for feature in features.mfcc:
#         feature_vector.append(np.mean(feature))
#         feature_vector.append(np.var(feature))
      
#       for feature in features.delta_mfcc:
#         feature_vector.append(np.mean(feature))
#         feature_vector.append(np.var(feature))

#       for feature in features.delta2_mfcc:
#         feature_vector.append(np.mean(feature))
#         feature_vector.append(np.var(feature))

#       feature_vector.append(np.mean(features.rmse))
#       feature_vector.append(np.var(features.rmse))

#       X = np.vstack((X,[feature_vector]))
#       if label == 'success':
#         Y = np.vstack((Y,[0,1]))
#         processed_counter += 1
#       elif label == 'stuttered':
#         Y = np.vstack((Y,[1,0]))
#         processed_counter += 1
#       else:
#         unexpected_label.append(file_path)
#       print("Done ", processed_counter, file_path)

# print("finished all!")    
# dataset = np.hstack((X,Y))

# if os.path.exists(output_file):
#   os.remove(output_file)

# np.savetxt(output_file, dataset)



      

Processing: stuttered
Done  1 fyp2-data-20201126-fortrain/stuttered/F_0101_13y1m_1-000162-um_augmented_fade_in_out.wav
Done  2 fyp2-data-20201126-fortrain/stuttered/M_0078_12y4m_1-000080-um_augmented_right_0.05_shifted.wav
Done  3 fyp2-data-20201126-fortrain/stuttered/M_0030_12y1m_1-000039-um_augmented_with_-100_gain.wav
Done  4 fyp2-data-20201126-fortrain/stuttered/M_0138_12y2m_1-000158-er_augmented_with_-100_gain.wav
Done  5 fyp2-data-20201126-fortrain/stuttered/M_0052_16y4m_1-000258-um_augmented_slowed.wav
Done  6 fyp2-data-20201126-fortrain/stuttered/M_0394_09y5m_1-000099-um.wav
Done  7 fyp2-data-20201126-fortrain/stuttered/M_0138_12y2m_1-000219-unk_augmented_0.9_toned.wav
Done  8 fyp2-data-20201126-fortrain/stuttered/M_0078_12y4m_1-000090-unk_augmented_fade_in_out.wav
Done  9 fyp2-data-20201126-fortrain/stuttered/M_0138_13y3m_1-000068-um_augmented_left_0.05_shifted.wav
Done  10 fyp2-data-20201126-fortrain/stuttered/M_0098_10y6m_1-000201-um_augmented_with_-50_gain.wav
Processing: s

All under is for graph purpose

In [ ]:
# def process(audio):
#   print(audio)
#   mfcc_object = FeatureExtraction()
#   mfcc_object.loadFile(f'{audio}')
#   mfcc_object.melspectrogram()
#   mfcc_object.plotmelspectrogram(True)
#   mfcc_object.extractmfcc()
#   mfcc_object.plotmfcc(True)

In [ ]:
#https://stackoverflow.com/questions/58068573/issue-with-pythons-multiprocessing-in-google-colab
#https://towardsdatascience.com/a-hands-on-guide-to-multiprocessing-in-python-48b59bfcc89e
# for audio in STUTTERED_AUDIOS:
#   print(audio)
#   mfcc_object = FeatureExtraction()
#   mfcc_object.loadFile(f'{audio}')
#   mfcc_objects.append(mfcc_object)

In [ ]:
# limit = 3
# counter = 0
# for mfcc_object in mfcc_objects:
#   if counter == limit:
#     break
#   mfcc_object.melspectrogram()
#   mfcc_object.plotmelspectrogram(True)
#   mfcc_object.extractmfcc()
#   mfcc_object.plotmfcc(True)
#   Audio(mfcc_object.filename)

In [ ]:
# !rm -rf mfcc
# !rm -rf mel

In [ ]:
# import multiprocessing
# import time
# import threading

In [ ]:
# Thread Example
# def task(i):
#     sema.acquire()
#     print( "start %s" % (i,))
#     time.sleep(2)
#     sema.release()

# maxthreads = 2
# sema = threading.Semaphore(value=maxthreads)
# threads = list()
# starttime = time.time()
# for i in range(10):
#     thread = threading.Thread(target=task,args=(i,))
#     threads.append(thread)
#     thread.start()
# print('Main thread done')
# print('Time taken = {} seconds'.format(time.time() - starttime))

In [ ]:
# def square(num):
#   square = num*num
#   print(square)

# def process(audio):
#   mfcc_object = FeatureExtraction()
#   mfcc_object.loadFile(audio)
#   mfcc_object.filename = audio.split('/')[3].split('.wav')[0]
#   mfcc_object.melspectrogram()
#   mfcc_object.plotmelspectrogram(save=True)
#   mfcc_object.extractmfcc()
#   mfcc_object.plotmfcc(save=True)
#   print(mfcc_object.filename)

# if __name__ == '__main__':
#   print("CPU Count: ", multiprocessing.cpu_count())
#   processes = []
#   starttime = time.time()
#   for audio in STUTTERED_AUDIOS:
#     p = multiprocessing.Process(target=process, args=(audio,))
#     processes.append(p)
#     p.start()
#     # process(audio)
  
#   for process in processes:
#     process.join()
#   print('Time taken = {} seconds'.format(time.time() - starttime))
# # def task(audio):
# #     sema.acquire()
# #     mfcc_object = FeatureExtraction()
# #     mfcc_object.loadFile(audio)
# #     mfcc_object.filename = audio.split('/')[3].split('.wav')[0]
# #     print( "process %s" % (mfcc_object.filename))
# #     mfcc_object.melspectrogram()
# #     mfcc_object.plotmelspectrogram(save=True)
# #     mfcc_object.extractmfcc()
# #     mfcc_object.plotmfcc(save=True)  
# #     time.sleep(2)
# #     sema.release()

# # maxthreads = 2
# # sema = threading.Semaphore(value=maxthreads)
# # threads = list()
# # starttime = time.time()
# # counter = 0
# # for audio in STUTTERED_AUDIOS:
# #     thread = threading.Thread(target=task,args=(audio,))
# #     threads.append(thread)
# #     thread.start()
# #     counter = counter + 1
# # print('Main thread done counter=',counter)
# # print('Time taken = {} seconds'.format(time.time() - starttime))

In [ ]:
# #https://stackoverflow.com/questions/53871991/how-to-download-folders-from-google-colaboratory
# import shutil
# shutil.make_archive('mel', 'zip', 'mel')
# shutil.make_archive('mfcc', 'zip', 'mfcc')

In [ ]:
# !rm -rf *.zip

In [ ]:
# !free -h
# !nvidia-smi -L
# !nvidia-smi
# !nvcc --version
# !lscpu | grep 'Model name'
# !lscpu | grep 'Core(s) per socket:'
# !lscpu | grep 'Threads(s) per core'
# mem = []
# while True:
#     mem.append(' ' * 10**6)
